In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

### Hyper params

In [2]:
mnist = input_data.read_data_sets('../dataset/MNIST_data', one_hot=True)

Extracting ../dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128

In [4]:
# X_side = int(np.sqrt(X_dim))
# DX_side = int(X_side/2)
# DX_dim = int(X_dim/4)

In [5]:
c = 0
lr = 1e-3

In [6]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

### Geneartor

In [7]:
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [8]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

### Discriminator

In [9]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])
D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [10]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

###  Training

In [11]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n]).astype(np.float32)

In [12]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

#### Generate data

In [13]:
def resize_image(sample, X_dim, X_side, DX_dim):
    dim0 = tf.shape(sample)[0]
    sample_reshaped = tf.reshape(sample, (dim0, X_side, X_side))
    sample_resized = tf.slice(sample_reshaped, 
                         [0, 0, 0], [dim0, DX_side, DX_side])
    sample_new = tf.reshape(sample_resized , [dim0, DX_dim])
    return sample_new

In [14]:
import random
wSet = np.zeros(X_dim).astype('float32')
r = random.sample(range(1, X_dim), 200)
wSet[r] = 1

In [15]:
Pw = tf.constant(wSet)

In [16]:
def pooling(sample, Pw):
    dim0 = tf.shape(sample)[0]
    B = tf.expand_dims(tf.ones([dim0]), 1) * Pw 
    sample_new = tf.multiply(sample, B)
    return sample_new

In [17]:
G_full_sample = generator(Z)
G_sample = pooling(G_full_sample, Pw)

In [18]:
D_real, D_logit_real = discriminator(X)

In [19]:
D_fake, D_logit_fake = discriminator(G_sample)

#### Loss

In [20]:
D_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake

G_gan_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
G_reg = tf.reduce_mean(tf.abs(G_full_sample)) 
alpha = 0.01
G_loss = G_gan_loss + alpha * G_reg

#### Optimize

In [21]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

#### Running

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [24]:
if not os.path.exists('out2/'):
    os.makedirs('out2/')

In [25]:
i = 0
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(G_full_sample, 
                   feed_dict={Z: sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out2/{}.png'.format(str(i).zfill(3)), 
                    bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_full, _ = mnist.train.next_batch(mb_size)
    dim0 = np.shape(X_full)[0]
    p = np.ones([dim0, 1]) * wSet 
    X_mb = np.multiply(X_full, p) 

    _, D_loss_curr = sess.run([D_solver, D_loss], 
                  feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], 
                  feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.364
G_loss: 1.519

Iter: 1000
D loss: 0.05676
G_loss: 5.61

Iter: 2000
D loss: 0.284
G_loss: 3.512

Iter: 3000
D loss: 0.589
G_loss: 2.21

Iter: 4000
D loss: 0.8708
G_loss: 1.923

Iter: 5000
D loss: 0.9289
G_loss: 1.935

Iter: 6000
D loss: 0.7484
G_loss: 1.748

Iter: 7000
D loss: 0.7836
G_loss: 2.072

Iter: 8000
D loss: 0.5342
G_loss: 1.898

Iter: 9000
D loss: 0.7057
G_loss: 2.896

Iter: 10000
D loss: 0.2874
G_loss: 3.104

Iter: 11000
D loss: 0.1295
G_loss: 3.919

Iter: 12000
D loss: 0.363
G_loss: 2.746

Iter: 13000
D loss: 0.3146
G_loss: 2.836

Iter: 14000
D loss: 0.6938
G_loss: 1.903

Iter: 15000
D loss: 0.9354
G_loss: 1.935

Iter: 16000
D loss: 0.9706
G_loss: 1.804

Iter: 17000
D loss: 0.9168
G_loss: 1.544

Iter: 18000
D loss: 0.9936
G_loss: 1.291

Iter: 19000
D loss: 1.013
G_loss: 1.277

Iter: 20000
D loss: 0.8552
G_loss: 1.337

Iter: 21000
D loss: 1.023
G_loss: 1.272

Iter: 22000
D loss: 0.8561
G_loss: 1.349

Iter: 23000
D loss: 1.13
G_loss: 1.309

Iter: 24000
D 

### Evaluation